<a href="https://colab.research.google.com/github/sunyingjian/AI-in-well-logging/blob/master/resnet_%E6%97%A0%E6%AD%A5%E9%95%BF_%E5%BB%B6%E5%AE%89%E6%B2%B9%E7%94%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#引入数据

In [ ]:
!git clone https://github.com/sunyingjian/numpy-.git

fatal: destination path 'numpy-' already exists and is not an empty directory.


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time

In [ ]:
%matplotlib inline
#%matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步。
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option
set_option("display.max_rows", 10)#设置要显示的默认行数，显示的最大行数是10
pd.options.mode.chained_assignment = None #为了在增加列表行数的时候防止出现setting with copy warning
training_data = pd.read_csv('/content/numpy-/3345train data.csv')
training_data
testing_data = pd.read_csv('/content/numpy-/3345test_data.csv')
testing_data

,AC,CAL,GR,K,RD,SP,Core Lithology
0,0.038756,0.026970,0.094776,0.112543,0.001420,0.614504,5
1,0.347382,0.090194,0.145150,0.202335,0.002744,0.339568,7
2,0.218400,0.059510,0.252165,0.711295,0.003874,0.918567,3
3,0.366064,0.082147,0.149304,0.201731,0.003290,0.343180,7
4,0.291877,0.079981,0.141782,0.289309,0.003455,0.344605,7
...,...,...,...,...,...,...,...
361,0.016947,0.138827,0.118400,0.358567,0.281687,0.302936,6
362,0.131043,0.069016,0.029354,0.060197,0.282093,0.253809,3
363,0.036182,0.032010,0.030090,0.071069,0.359737,0.435486,5
364,0.014562,0.127332,0.035014,0.178579,0.803566,0.311641,6


##smote

In [ ]:
a = training_data['Core Lithology'].value_counts()
a

7    928
3    751
2    710
5    482
1    194
4    166
6    114
Name: Core Lithology, dtype: int64

In [ ]:
X_train = training_data.drop(columns='Core Lithology')
y_train = training_data['Core Lithology'].values

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE(sampling_strategy={5:700,1:700,4:700,6:700},random_state=24)
X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)

train_x_sm = pd.DataFrame(X_train_sm)
train_y_sm = pd.DataFrame(y_train_sm)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
train_x_sm['Core Lithology'] = y_train_sm
training_data_sm = train_x_sm

In [ ]:
X_train_sm = training_data_sm.drop(columns='Core Lithology')
y_train_sm = training_data_sm['Core Lithology'].values

In [ ]:
X_train_sm.shape

(5189, 6)

In [ ]:
#升维
X_train_sm = tf.expand_dims(X_train_sm,1)

In [ ]:
train_x = X_train_sm

In [ ]:
X_test = testing_data.drop(columns='Core Lithology')

In [ ]:
y_test = testing_data['Core Lithology'].values

In [ ]:
X_test = tf.expand_dims(X_test, 1 )

In [ ]:
test_x = X_test

In [ ]:
#onehot编码
y_test = y_test-1
y_train_sm = y_train_sm-1

In [ ]:
train_y = tf.keras.utils.to_categorical(y_train_sm,7)

In [ ]:
test_y = tf.keras.utils.to_categorical(y_test,7)

##ResNet

In [ ]:
from tensorflow import keras
import math
import tensorflow_addons as tfa
import tensorflow as tf

In [ ]:
input_layer = tf.keras.Input(shape=(1,6))
n_feature_maps = 16
attention = 6

In [ ]:
# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, strides=attention,padding='same')(input_layer)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, strides=attention, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, strides=attention, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, strides=attention, kernel_size=1, padding='same')(input_layer)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, strides=attention*2, kernel_size=8, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, strides=attention*2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, strides=attention*2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, strides=attention*2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
output_block_2 = keras.layers.Activation('relu')(output_block_2)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, strides=attention*2, kernel_size=8, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, strides=attention*2, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, strides=attention*2, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)

# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

output_layer = keras.layers.Dense(7, activation='softmax')(gap_layer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 6)]       0                                            
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 1, 16)        784         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 1, 16)        64          conv1d_33[0][0]                  
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 1, 16)        0           batch_normalization_36[0][0]     
_______________________________________________________________________________________

In [ ]:
#学习率衰减
learning_rate=0.001
def scheduler(epoch):
  if epoch<=200:
    learning_rate=0.001
  elif epoch<=300:
    learning_rate=0.0008
  elif epoch<=400:
    learning_rate=0.0006
  else:
    learning_rate=0.0005
  return learning_rate
  
Lr_change=keras.callbacks.LearningRateScheduler(scheduler)
#保存准确率最好的模型
from keras.callbacks import ModelCheckpoint
filepath="/content/drive/My Drive/best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max')

#早停止
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

#Adam优化器
Adam=keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=Adam,loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history=model.fit(train_x,train_y,batch_size=128 ,
         epochs=100,  
         callbacks=[Lr_change,checkpoint,callback],
         validation_data=(test_x,test_y))

Epoch 1/100
41/41 [==============================] - ETA: 0s - loss: 1.5784 - acc: 0.4407
Epoch 00001: val_acc improved from -inf to 0.20219, saving model to /content/drive/My Drive/best_weight.h5
41/41 [==============================] - 1s 17ms/step - loss: 1.5784 - acc: 0.4407 - val_loss: 1.9469 - val_acc: 0.2022
Epoch 2/100
36/41 [=========================>....] - ETA: 0s - loss: 1.0584 - acc: 0.6354
Epoch 00002: val_acc improved from 0.20219 to 0.20492, saving model to /content/drive/My Drive/best_weight.h5
41/41 [==============================] - 0s 8ms/step - loss: 1.0389 - acc: 0.6433 - val_loss: 1.9773 - val_acc: 0.2049
Epoch 3/100
41/41 [==============================] - ETA: 0s - loss: 0.8628 - acc: 0.7105
Epoch 00003: val_acc did not improve from 0.20492
41/41 [==============================] - 0s 9ms/step - loss: 0.8628 - acc: 0.7105 - val_loss: 2.0215 - val_acc: 0.1776
Epoch 4/100
36/41 [=========================>....] - ETA: 0s - loss: 0.7662 - acc: 0.7331
Epoch 00004: va